Run spliceAI on all of the genes. Save the numeric output for splicing probability

For each transcript, save a value for each window of size X.
  0 is not a splice junction
  1 is a perfect end of the exon
  2 contains the splicing junction
  3 is a perfect start of an exon

Generate a vector which puts three strings under a the fullSeq                     ATAGGCGCGTAGATCGATGCATAGGA
One that maps the seq to a series of window frames of length X                     ------0123456789....
The next is the distance between the window's right edge and the next exon end     ------9876543210XXXXXXXX98
And the other is distance to next exon start                                       ------XXXXXXXX9876543210XX


Parallels this work:
https://www.matreyeklab.com/using-spliceai-for-synthetic-constructs/957/

In [ ]:
!pip install -q condacolab
import condacolab
condacolab.install()

⏬ Downloading https://github.com/conda-forge/miniforge/releases/download/23.11.0-0/Mambaforge-23.11.0-0-Linux-x86_64.sh...
📦 Installing...
📌 Adjusting configuration...
🩹 Patching environment...
⏲ Done in 0:00:17
🔁 Restarting kernel...


In [ ]:
!pip install refgene_parser
!pip install pyfastx
#!pip install spliceai
#!pip install biopython

import sys
import condacolab
condacolab.check()
!conda init
#!conda update -n base -c conda-forge conda
!conda install python=3.7
!conda create --name spliceaienv python=3.7 #3.6 #3.7
#!conda init
!conda activate spliceaienv
#!conda activate base
#add channels
!conda install -c conda-forge keras #==2.0.5
!conda install -c conda-forge h5py #==2.10.0
!conda install -c bioconda spliceai
!conda install -c bioconda biopython
#!conda install tensorflow
#!conda install -c conda-forge tensorflow #==1.9.0 #tensorflow>=2.16.0
!conda install -c conda-forge pandas #==0.24.2
!conda install -c conda-forge openssl #==3.2.0
#!conda install -c conda-forge keras==2.3.1
!conda install nomkl
!conda install numpy #==1.26.4
#!conda install mkl-service

!pip install tensorflow

print("###########")
print(sys.version)
print("###########")

from google.colab import drive
drive.mount('/content/drive')
%cd drive
%cd MyDrive
%ls
from GeneClasses import NaturalGene, ProteinObj, Isoform, IsoformGeneBody

from refgene_parser import RefGene
import pyfastx
import tensorflow as tf
print(tf.__version__)
import spliceai
import pandas as pd
import keras
import numpy as np
import os
import dataclasses
import json
#from tf.keras.models import load_model
from keras.models import load_model
from pkg_resources import resource_filename
from spliceai.utils import one_hot_encode


✨🍰✨ Everything looks OK!
no change     /usr/local/condabin/conda
no change     /usr/local/bin/conda
no change     /usr/local/bin/conda-env
no change     /usr/local/bin/activate
no change     /usr/local/bin/deactivate
no change     /usr/local/etc/profile.d/conda.sh
no change     /usr/local/etc/fish/conf.d/conda.fish
no change     /usr/local/shell/condabin/Conda.psm1
no change     /usr/local/shell/condabin/conda-hook.ps1
no change     /usr/local/lib/python3.10/site-packages/xontrib/conda.xsh
no change     /usr/local/etc/profile.d/conda.csh
modified      /root/.bashrc

==> For changes to take effect, close and re-open your current shell. <==

Channels:
 - conda-forge
Platform: linux-64
Solving environment: / failed

SpecsConfigurationConflictError: Requested specs conflict with configured specs.
  requested specs: 
    - python=3.7
  pinned specs: 
    - cuda-version=12
    - python=3.10
    - python_abi=3.10[build=*cp310*]
Use 'conda config --show-sources' to look for 'pinned_specs' an

Get a list of the genes we'll work with

In [ ]:
#Get the list of gene objects
def listGeneObjs():
  root = r'\RefGenes\NHGeneBodySupp'
  #root = os.getcwd()
  files = []
  for r, d, f in os.walk(os.path.join(os.getcwd(), "RefGenes", "NHGeneBodySupp")):
      for file in f:
          #print(file)
          if '.json' in file:
              #print(os.path.join(r, file))
              files.append(os.path.join(r, file))
  return files

genes = listGeneObjs()
#print(genes)

One by one, load the gene objects, run spliceAI on them, and run CURVES+ and/or canal/biobb on the primary transcript

In [ ]:
#Load each in sequence, and run spliceAI on the transcript, saving a new gene object in place of the old one with spliceAI donor and acceptor scores
def spliceAIOnPrimaryTranscript(gene, contextint: int):
  padSeq = None
  if len(gene.DNASequence) < contextint:
    NPad = 3000 - len(gene.DNASequence)
    LPad = int(NPad/2) + (NPad % 2) # + 1
    RPad = 'N' * LPad
    LPad = RPad
    padSeq = LPad + gene.DNASequence + RPad
  else:
    padSeq = gene.DNASequence

  input_sequence = padSeq
  #input_sequence = gene.DNASequence
  # Replace this with your custom sequence
            #10000
            #3000
  #context = len(padSeq)
  context = contextint
  paths = ('models/spliceai{}.h5'.format(x) for x in range(1, 6))
  models = [load_model(resource_filename('spliceai', x), compile=False) for x in paths]
  x = one_hot_encode('N'*(context//2) + input_sequence + 'N'*(context//2))[None, :]
  y = np.mean([models[m].predict(x) for m in range(5)], axis=0)

  acceptor_prob = y[0, :, 1]
  donor_prob = y[0, :, 2]

  return acceptor_prob, donor_prob


In [ ]:
def loadGeneBody(gne):
  gene = None

  with open(gne, 'r') as infile:
      gene = json.load(infile)
  if gene is None:
    print("Gene not found")
  isosList = gene['isoforms']
  geneID = gene['geneID']
  geneName = gene['geneName']
  organism = gene['organism']
  DNASequence = gene['DNASequence']
  spliceAIDonor = gene['spliceAIDonor']
  spliceAIReceptor = gene['spliceAIReceptor']
  ######3
  chromosome = gene['chromosome']

  isoformsObjs = []
  for iso in isosList:
    isoformNumber = iso['isoformNumber']
    #
    associatedProteinPC = iso['associatedProtein']
    aaSeq = associatedProteinPC['aaSeq']
    protWeight = associatedProteinPC['protWeight']
    associatedGeneID = associatedProteinPC['associatedGeneID']
    associatedProtein = ProteinObj(aaSeq, protWeight, associatedGeneID)
    #
    fullSequence = iso['fullSequence']
    codingSeq = iso['codingSeq']
    relativeAbundance = iso['relativeAbundance']
    geneBody = iso['geneBody']
    isoformsObjs.append(IsoformGeneBody(isoformNumber, associatedProtein, fullSequence, codingSeq, relativeAbundance, geneBody))
  geneInQuestion = NaturalGene(isoformsObjs, geneID, geneName, organism, DNASequence, spliceAIDonor, spliceAIReceptor, {}, chromosome)
  return geneInQuestion



"""

@dataclass
class ProteinObj:
    aaSeq: str
    protWeight: float
    associatedGeneID: int

@dataclass
class NaturalGene:
    isoforms: list
    geneID: int
    geneName: str
    organism: str
    DNASequence: str
    spliceAIDonor: list
    spliceAIReceptor: list
    chromosome: int

@dataclass
class IsoformGeneBody:
    isoformNumber: str
    associatedProtein: ProteinObj
    fullSequence: str
    codingSeq: str
    relativeAbundance: float
    geneBody: list

@dataclass
class Isoform:
    isoformNumber: str
    associatedProtein: ProteinObj
    fullSequence: str
    codingSeq: str
    relativeAbundance: float

"""

def saveNaturalGeneObj(obj: NaturalGene):

    print("CWD: ", str(os.getcwd()))
    #while "MyDrive" in os.getcwd():
    #  %cd ..
    #%cd MyDrive
    #%cd RefGenes
    #%cd NHGeneBodySupp

    name = str(obj.geneID) + r'.json'
    #root = r'RefGenes/NHGenes'
    #fname = os.path.join(root, name)
    fname = name
    print(fname)
    print("Beginning json dumps of natural gene")
    for iso in obj.isoforms:
      assocProtJSON = dataclasses.asdict(iso.associatedProtein)
      #for exon in iso.geneBody:
      #  exonJSON = dataclasses.asdict(exon)
      print("Can we do it?")
      isoAsDict = dataclasses.asdict(iso)
      print(isoAsDict)
      #isoAsDict['associatedProtein'] = assocProtJSON
      #isoAsDict['geneBody'] = exonJSON


    #raise NotImplementedError

    b = json.dumps(dataclasses.asdict(obj))
    print(b)
    with open(fname, 'wb') as infile:
        infile.write(b.encode('utf-8'))
        #Findme
    infile.close()
    print("# Closed")
    #%cd ..
    #%cd ..
    #%cd WorkingFolders
    #%cd NCBIDownload


"""
def saveNaturalGeneObj(obj: NaturalGene):
    print("CWD: ", str(os.getcwd()))
    #while "MyDrive" in os.getcwd():
    #  %cd ..
    #%cd MyDrive
    #%cd RefGenes
    #%cd NHGeneBodySupp

    name = os.path.join("firstPassPlusSpliceAI", str(obj.geneID) + '.json')
    #root = r'RefGenes/NHGenes'
    #fname = os.path.join(root, name)
    fname = name
    print(fname)
    #raise NotImplementedError

    with open(fname, 'w') as infile:
        #j = json.dumps(obj, default=lambda obj: obj.__dict__)
        j = json.dumps(dataclasses.asdict(obj))
        infile.write(j.encode('utf-8'))
        #Findme
    infile.close()

    #%cd ..
    #%cd ..
    #%cd WorkingFolders
    #%cd NCBIDownload
"""

def delGene(geneObj: NaturalGene):
  print("CWD: ", str(os.getcwd()))
  name = os.path.join("RefGenes", "NHGeneBodySupp", str(geneObj.geneID) + r'.json')
  print(name)
  raise NotImplementedError
  os.remove(name)
  print("#deleted")


In [ ]:
print(genes)
for g in genes:
  gene = loadGeneBody(g)
  gene.spliceAIReceptor, gene.spliceAIDonor = spliceAIOnPrimaryTranscript(gene, 10000)
  gene.spliceAIReceptor = gene.spliceAIReceptor.tolist()
  gene.spliceAIDonor = gene.spliceAIDonor.tolist()
  saveNaturalGeneObj(gene)
  delGene(gene)




NameError: name 'genes' is not defined

In [ ]:
"""
from keras.models import load_model
from pkg_resources import resource_filename
from spliceai.utils import one_hot_encode
import numpy as np

input_sequence = 'CGATCTGACGTGGGTGTCATCGCATTATCGATATTGCAT'
# Replace this with your custom sequence

context = 10000
paths = ('models/spliceai{}.h5'.format(x) for x in range(1, 6))
models = [load_model(resource_filename('spliceai', x)) for x in paths]
x = one_hot_encode('N'*(context//2) + input_sequence + 'N'*(context//2))[None, :]
y = np.mean([models[m].predict(x) for m in range(5)], axis=0)

acceptor_prob = y[0, :, 1]
donor_prob = y[0, :, 2]
"""

In [ ]:
"""
for natGene in genes:
  gene = None
  with open(natGene, 'rb') as infile:
    gene = json.loads(infile)
    print(gene)

  gene.spliceAIReceptor, gene.spliceAIDonor = spliceAIOnPrimaryTranscript(gene)
  root = r'\RefGenes\NHGeneBodySupp'
  with open(natGene, 'wb') as outfile:
    pickle.dump(gene, outfile)
"""